## Library Imports

In [11]:
import numpy as np
import cv2
from sklearn.preprocessing import StandardScaler
from PIL import Image
import os
import random

## Image Matrix Construction

In [12]:
def create_vec(location, count_of_same_celebrity=40):
    '''
    Create a random vectorized array with 'n' number of images of each celebrity

    location: location of the master folder
    count_of_same_celebrity: number of random images taken of the same celebrity
    '''
    # Create an empty list to store arrays
    img_arrays = []

    for celeb in os.listdir(location):
        celeb_path = os.path.join(location, celeb)
        if not os.path.isdir(celeb_path):
            continue  # Skip if the item in the directory is not a folder

        files = [f for f in os.listdir(celeb_path) if os.path.isfile(os.path.join(celeb_path, f))]

        # Use min() to avoid going beyond the available indices
        selected_files = random.sample(files, min(len(files), count_of_same_celebrity))

        for selected_file in selected_files:
            face_path = os.path.join(celeb_path, selected_file)

            # Open and resize the face image
            face_image = Image.open(face_path).convert("L")  # Convert to grayscale
            resized_face_image = face_image.resize((20, 20))
            img_array = np.array(resized_face_image).flatten()
            img_arrays.append(img_array)

    # Convert the list of arrays into a NumPy array
    img_matrix = np.transpose(img_arrays)

    return img_matrix


A = create_vec('Cropped Celebrity Faces Dataset')

In [13]:
n = A.shape[0]
m = A.shape[1]
print("n =", n)
print("m =", m)

n = 400
m = 680


In [14]:
mean = np.mean(A, axis=1, keepdims=True)
B = A - mean

## Covariance Matrix Generation

In [15]:
S = 1/m * B @ B.T
S.shape

(400, 400)

## Zhang et al. Encryption

In [16]:
from scipy.sparse import rand

alpha = np.random.rand()
gamma = np.random.rand()
G = np.triu(np.random.rand(n, n), 0)

S1 = alpha * S + gamma * np.eye(n)
U = np.block([[S1, np.zeros((n,n))], [np.zeros((n,n)), G]])

P = np.eye(2 * n)

for k in range(1, 2 * n + 1):
    C = np.eye(2 * n)
    j = np.random.randint(2 * n - 1)
    i = np.random.randint(j + 1, 2 * n)
    C[i, j] = np.random.rand()
    P = np.dot(P, C)


U1 = P @ U @ np.linalg.inv(P)
U1.shape

(800, 800)

## Eigendecomposition 

In [17]:
# D=Eigenvalues V=Eigenvectors
D , V = np.linalg.eig(U1)
D1=np.diag(D)
D1.shape

(800, 800)

## Verification

In [18]:
l=int(input('No. of trials'))

for i in range(l):
    e = np.random.choice([0, 1], size=(V.shape[0]))
    n2 = np.linalg.norm(V @ (D1 @ (V.T @ e)) - U1 @ e)
    
    if n2 >= 1e-4:
        print("Verification failed")
        break
    
else:
    print("Verification successful for all trials")

No. of trials1
Verification failed


### Error Calculation 

In [19]:
error1 = np.linalg.norm(V @ D1 @ V.T - U1)
error1

308210.477636223

## Zhang et al. Encryption Upon Correction

### Generation of U

In [20]:
alpha1 = np.random.rand()
gamma1 = np.random.rand()

G1 = np.diag(np.random.rand(n))
S2 = alpha1 * S + gamma1 * np.eye(n)
U2 = np.block([[S2, np.zeros((n,n))], [np.zeros((n,n)), G1]])

# Multiplying orthogonal matrices
from scipy.sparse import rand
u1 = rand(2*S.shape[0], 1);
u1 = u1.toarray();
u1 = u1 / np.linalg.norm(u1)
P1 = np.identity(2*S.shape[0]) - 2 * (u1 @ u1.T)
U3 = P1 @ U2 @ P1.T

### Eigendecomposition 

In [24]:
# D=Eigenvalues V=Eigenvectors
D2 , V1 = np.linalg.eig(U3)
D3=np.diag(D2)
D3.shape

(800, 800)

### Verification

In [25]:
l=int(input('No. of trials'))

for i in range(l):
    e1 = np.random.choice([0, 1], size=(V.shape[0]))
    n1 = np.linalg.norm(V1 @ (D3 @ (V1.T @ e1)) - U3 @ e1)
    
    if n1 >= 1e-4:
        print("Verification failed")
        break
    
else:
    print("Verification successful for all trials")

No. of trials80
Verification successful for all trials


### New Error Calculation

In [26]:
error2 = np.linalg.norm(V1 @ D3 @ V1.T - U3)
error2

1.9891968786295523e-09